In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_dir = "..\\data"

## Read data

In [3]:
def read_ff(filename, market):
    """
    Read ff portfolios
    """
    df = pd.read_excel(os.path.join(data_dir, filename), na_values=[-99.99])
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m').dt.to_period('M')
    df = df.set_index('Date')
    if market == 'EM':
        df = df.loc['1991-07':'2019-12']
    else:
        df = df.loc['1990-07':'2019-12']
    return df
'''
def read_qmj(filename):
    """
    Read QMJ (AQR factor)
    """
    df = pd.read_excel(os.path.join(data_dir, filename))
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y').dt.to_period('M')
    df = df.set_index('Date')
    df = df.loc['1993-07':'2019-12']
    return df
'''

def describe(df, n=2):
    print(df.shape)
    display(df.head(n))
    display(df.tail(n))

## Regression Logic

In [4]:
def run_regression(y, X=None):
    """
    Run regression based on X and y
    """
    if X is not None:
        X = sm.add_constant(X) 
    else:
        X = np.ones((len(y), 1))
        
    model = sm.OLS(endog=y, exog=X).fit()
    coeff = model.params.values
    tvalues = model.tvalues.values
    
    if X.shape[1] == 1:
        return '{0:.2f}'.format(coeff[0]), '({0:.2f})'.format(tvalues[0])
    
    return ['{0:.2f}'.format(x) for x in coeff], ['({0:.2f})'.format(x) for x in tvalues]


def get_exog(df, col):
    """
    Prepare X dataframe as per given column
    """
    if col is 'alpha':
        return None
    elif col == 'Mkt':
        return df[['Mkt-RF']]
    elif col == 'Mkt(-1)':
        X = df[['Mkt-RF']]
        X['Mkt-RF(-1)'] = X['Mkt-RF'].shift(1)
        X = X.dropna()
        return X
    elif col == 'WML':
        X = df[['Mkt-RF', 'HML', 'WML']]
        X['Mkt-RF(-1)'] = X['Mkt-RF'].shift(1)
        X = X.dropna()
        X = X.reindex(columns=['Mkt-RF', 'Mkt-RF(-1)', 'HML', 'WML'])
        return X
    elif col == 'CMA':
        X = df[['Mkt-RF', 'HML', 'WML', 'RMW', 'CMA']]
        X['Mkt-RF(-1)'] = X['Mkt-RF'].shift(1)
        X = X.dropna()
        X = X.reindex(columns=['Mkt-RF', 'Mkt-RF(-1)', 'HML', 'WML', 'RMW', 'CMA'])
        return X

In [17]:
def get_exhibit(ff3, ff5_wml, market_name=''):
    """
    Get exhibit results for a particular market
    """
    # Exhibit template
    exhibit = pd.DataFrame(columns=['alpha', 'Mkt', 'Mkt(-1)', 'HML', 'WML', 'RMW', 'CMA'],
                        index=pd.MultiIndex.from_product([[1, 2, 3, 4, 5], ['coeff.', 't-stat.']], names=['', market_name]))
    exhibit = exhibit.fillna('-')
    
    # Fill up exhibit template
    endogs = [['alpha'], 
              ['alpha', 'Mkt'],
              ['alpha', 'Mkt', 'Mkt(-1)'],
              ['alpha', 'Mkt', 'Mkt(-1)', 'HML', 'WML'], 
              ['alpha', 'Mkt', 'Mkt(-1)', 'HML', 'WML', 'RMW', 'CMA']]

    for idx, cols in enumerate(endogs, 1):
        y = ff3['SMB']
        end_col = cols[-1]
        if end_col == 'alpha':
            coeff, tvalues = run_regression(y=y, X=None)
            exhibit.loc[(idx, 'coeff.'), cols] = coeff
            exhibit.loc[(idx, 't-stat.'), cols] = tvalues
        elif end_col == 'Mkt':
            X = get_exog(df=ff3, col=end_col)
            y = y.loc[X.index]
            coeff, tvalues = run_regression(y=y, X=X)
            exhibit.loc[(idx, 'coeff.'), cols] = coeff
            exhibit.loc[(idx, 't-stat.'), cols] = tvalues
        elif end_col == 'Mkt(-1)':
            X = get_exog(df=ff3, col=end_col)
            y = y.loc[X.index]
            coeff, tvalues = run_regression(y=y, X=X)
            exhibit.loc[(idx, 'coeff.'), cols] = coeff
            exhibit.loc[(idx, 't-stat.'), cols] = tvalues
        elif end_col == 'WML':
            X = get_exog(df=ff5_wml, col=end_col)
            y = y.loc[X.index]
            coeff, tvalues = run_regression(y=y, X=X)
            exhibit.loc[(idx, 'coeff.'), cols] = coeff
            exhibit.loc[(idx, 't-stat.'), cols] = tvalues
        elif end_col == 'CMA':
            X = get_exog(df=ff5_wml, col=end_col)
            y = y.loc[X.index]
            coeff, tvalues = run_regression(y=y, X=X)
            exhibit.loc[(idx, 'coeff.'), cols] = coeff
            exhibit.loc[(idx, 't-stat.'), cols] = tvalues
            
    return exhibit

## Europe

In [18]:
# FF3 - Europe
ff3_eu = read_ff("FF3_EU.xlsx", market='EU')

# FF5 - Europe
ff5_eu = read_ff("FF5_EU.xlsx", market='EU')

# WML a.k.a Momentum - Europe
wml_eu = read_ff("WML_EU.xlsx", market='EU')

# FF5 + Momentum - Europe
# Note: Momentum factor in EU only starts from Nov 1990, unlike other which start July 1990
ff5_wml_eu = wml_eu.join(ff5_eu)

In [19]:
exhibit2_eu = get_exhibit(ff3=ff3_eu, ff5_wml=ff5_wml_eu, market_name='Europe')
exhibit2_eu

alpha      Mkt Mkt(-1)      HML     WML      RMW      CMA
  Europe                                                              
1 coeff.     -0.01        -       -        -       -        -        -
  t-stat.  (-0.08)        -       -        -       -        -        -
2 coeff.      0.03    -0.07       -        -       -        -        -
  t-stat.   (0.23)  (-3.06)       -        -       -        -        -
3 coeff.     -0.04    -0.09    0.15        -       -        -        -
  t-stat.  (-0.39)  (-3.88)  (6.87)        -       -        -        -
4 coeff.     -0.04    -0.08    0.16    -0.05    0.01        -        -
  t-stat.  (-0.36)  (-3.18)  (7.07)  (-1.10)  (0.46)        -        -
5 coeff.      0.02    -0.09    0.16    -0.07    0.03    -0.14    -0.05
  t-stat.   (0.17)  (-3.32)  (6.86)  (-1.02)  (0.93)  (-1.58)  (-0.57)

## Asia-Pacific ex-Japan

In [20]:
# FF3 - Asia-Pacific ex-Japan
ff3_apej = read_ff("FF3_APeJ.xlsx", market='APeJ')

# FF5 - Asia-Pacific ex-Japan
ff5_apej = read_ff("FF5_APeJ.xlsx", market='APeJ')

# WML a.k.a Momentum - Asia-Pacific ex-Japan
wml_apej = read_ff("WML_APeJ.xlsx", market='APeJ')

# FF5 + Momentum - Asia-Pacific ex-Japan
ff5_wml_apej = wml_apej.join(ff5_apej)
describe(ff5_wml_apej)

(350, 7)


,WML,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
1990-11,2.77,-2.98,-1.58,-0.77,2.65,0.56,0.57
1990-12,2.51,-1.10,-2.74,-1.02,0.28,-1.49,0.60


,WML,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
2019-11,-0.75,-0.20,-1.48,-0.79,2.05,-0.4,0.12
2019-12,1.90,2.99,1.47,1.21,0.47,-1.0,0.14


In [21]:
exhibit2_apej = get_exhibit(ff3=ff3_apej, ff5_wml=ff5_wml_apej, market_name='Asia-Pacific ex-Japan')
exhibit2_apej

alpha      Mkt Mkt(-1)      HML     WML      RMW  \
  Asia-Pacific ex-Japan                                                       
1 coeff.                   -0.29        -       -        -       -        -   
  t-stat.                (-1.85)        -       -        -       -        -   
2 coeff.                   -0.28    -0.01       -        -       -        -   
  t-stat.                (-1.79)  (-0.43)       -        -       -        -   
3 coeff.                   -0.36    -0.03    0.16        -       -        -   
  t-stat.                (-2.43)  (-1.11)  (6.03)        -       -        -   
4 coeff.                   -0.40    -0.01    0.16    -0.07    0.05        -   
  t-stat.                (-2.50)  (-0.42)  (6.27)  (-1.27)  (1.40)        -   
5 coeff.                   -0.12    -0.08    0.15    -0.22    0.06    -0.35   
  t-stat.                (-0.73)  (-2.71)  (5.96)  (-3.27)  (1.65)  (-4.67)   

                             CMA  
  Asia-Pacific ex-Japan           
1 coeff.                       -  
  t-stat.                      -  
2 coeff.                       -  
  t-stat.                      -  
3 coeff.                       -  
  t-stat.                      -  
4 coeff.                       -  
  t-stat.                      -  
5 coeff.                   -0.11  
  t-stat.                (-1.45)

## Japan

In [22]:
# FF3 - Japan
ff3_jpn = read_ff("FF3_JPN.xlsx", market='JPN')
describe(ff3_jpn)

# FF5 - Japan
ff5_jpn = read_ff("FF5_JPN.xlsx", market='JPN')

# WML a.k.a Momentum - Japan
wml_jpn = read_ff("WML_JPN.xlsx", market='JPN')

# FF5 + Momentum - Japan
ff5_wml_jpn = wml_jpn.join(ff5_jpn)
describe(ff5_wml_jpn)

(354, 4)


,Mkt-RF,SMB,HML,RF
Date,,,,
1990-07,0.10,6.00,3.63,0.68
1990-08,-11.88,-4.97,0.33,0.66


,Mkt-RF,SMB,HML,RF
Date,,,,
2019-11,0.67,1.53,-1.97,0.12
2019-12,2.07,0.75,0.94,0.14


(350, 7)


,WML,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
1990-11,1.50,-14.12,-5.20,-0.20,3.86,-2.18,0.57
1990-12,-8.04,1.93,-6.11,-3.61,1.23,1.87,0.60


,WML,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
2019-11,0.53,0.67,1.31,-1.97,0.16,-0.80,0.12
2019-12,0.79,2.07,0.92,0.94,-0.24,-1.18,0.14


In [23]:
exhibit2_jpn = get_exhibit(ff3=ff3_jpn, ff5_wml=ff5_wml_jpn, market_name='Japan')
exhibit2_jpn

alpha     Mkt Mkt(-1)      HML      WML      RMW     CMA
  Japan                                                             
1 coeff.     0.05       -       -        -        -        -       -
  t-stat.  (0.30)       -       -        -        -        -       -
2 coeff.     0.04    0.08       -        -        -        -       -
  t-stat.  (0.26)  (2.57)       -        -        -        -       -
3 coeff.     0.02    0.07    0.09        -        -        -       -
  t-stat.  (0.12)  (2.40)  (3.02)        -        -        -       -
4 coeff.     0.06    0.04    0.11    -0.06    -0.04        -       -
  t-stat.  (0.37)  (1.36)  (3.56)  (-0.96)  (-1.06)        -       -
5 coeff.     0.09    0.03    0.13    -0.22    -0.02    -0.00    0.37
  t-stat.  (0.53)  (0.98)  (4.23)  (-3.11)  (-0.55)  (-0.01)  (3.59)

## Emerging Markets

In [24]:
# FF5 - Emerging Markets
ff5_em = read_ff("FF5_EM.xlsx", market='EM')

# FF3 - Emerging Markets
# Subsetting from FF5 data since EM data is not available on FF website
ff3_em = ff5_em[['Mkt-RF', 'SMB', 'HML', 'RF']]

# WML a.k.a Momentum - Emerging Markets
wml_em = read_ff("WML_EM.xlsx", market='EM')

# FF5 + Momentum - Emerging Markets
ff5_wml_em = wml_em.join(ff5_em)
describe(ff5_wml_em)

(342, 7)


,WML,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
1991-07,0.40,0.90,-6.29,2.89,-1.3,NaN,0.49
1991-08,0.02,0.11,-4.11,2.45,2.8,NaN,0.46


,WML,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
2019-11,0.22,-0.4,-1.6,0.87,-0.56,-1.64,0.12
2019-12,0.28,6.8,-0.7,0.27,2.60,-0.72,0.14


In [25]:
exhibit2_em = get_exhibit(ff3=ff3_em, ff5_wml=ff5_wml_em, market_name='Emerging Markets')
exhibit2_em

alpha      Mkt Mkt(-1)      HML     WML      RMW  \
  Emerging Markets                                                       
1 coeff.               0.03        -       -        -       -        -   
  t-stat.            (0.23)        -       -        -       -        -   
2 coeff.               0.09    -0.09       -        -       -        -   
  t-stat.            (0.75)  (-4.80)       -        -       -        -   
3 coeff.               0.05    -0.11    0.10        -       -        -   
  t-stat.            (0.49)  (-6.06)  (5.52)        -       -        -   
4 coeff.              -0.00    -0.10    0.11    -0.05    0.10        -   
  t-stat.           (-0.01)  (-5.43)  (5.75)  (-0.95)  (2.49)        -   
5 coeff.               0.18    -0.12    0.09    -0.15    0.14    -0.45   
  t-stat.            (1.55)  (-6.37)  (5.34)  (-2.53)  (3.76)  (-5.35)   

                        CMA  
  Emerging Markets           
1 coeff.                  -  
  t-stat.                 -  
2 coeff.                  -  
  t-stat.                 -  
3 coeff.                  -  
  t-stat.                 -  
4 coeff.                  -  
  t-stat.                 -  
5 coeff.              -0.10  
  t-stat.           (-1.57)